In [1]:
import pandas as pd
import numpy as np

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" ##url for table

tables = pd.read_html(url, index_col=0, attrs={"class":"wikitable"}) ##gets table
df = tables[0] ##assigns name to table
df.shape

(287, 2)

In [3]:
indexNames = df[ df['Borough'] == 'Not assigned' ].index ## rows with a borough equal to NA
df.drop(indexNames , inplace=True) ## deletes the rows found in the above line
df.reset_index(inplace=True) ##makes the index the row number, not the postcode
df.rename(columns={"Postcode":"PostalCode", "Neighbourhood":"Neighborhood"}, inplace = True)##renames columns
df.shape

(210, 3)

In [4]:
df['Neighborhood'] = np.where(df['Neighborhood'] == "Not assigned", df['Borough'], df['Neighborhood']) ##renames "Not assigned" neighborhood values

The below code selects the three columns to work on, groups by postal code and borough, and then selects all the neighborhoods that match that combination. The second line deletes the duplicate rows since this gets applied to every row.

In [5]:
df['Neighborhood'] = df[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df.drop_duplicates(subset=['PostalCode','Borough','Neighborhood'], keep="first", inplace=True)
df = df.reset_index(drop=True)
df.head(10)##check to make sure work as expected

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [6]:
df.shape ##Answer for Part 1

(103, 3)

In [7]:
df_csv = pd.read_csv("https://cocl.us/Geospatial_data") ##load CSV data

In [8]:
df_csv.head() ##look at csv data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_csv.rename(columns={"Postal Code":"PostalCode"}, inplace = True)##renames columns to match naming in first dataframe

In [10]:
df_final = df.join(df_csv.set_index('PostalCode'), on='PostalCode') ##joins the two datatables using the key "Postal Code"

In [11]:
df_final.head() ##answer for part 2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [12]:
import json ##import everything
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

# All requested packages already installed.



In [13]:
# The code was removed by Watson Studio for sharing.

I decided to include only the neighborhoods in Boroughs containing Toronto to minimize the number of calls. I'm assuming the postal codes are what we are using as "neighborhoods" since it's the unique feature being used in both the above steps

In [14]:
Toronto_data = df_final[df_final['Borough'].isin(['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto'])].reset_index(drop=True) ##selects postal codes in the toronto boroughs

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): ##function like in New york to run through neighborhoods
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Toronto_venues = getNearbyVenues(names=Toronto_data['PostalCode'], 
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  ) ##runs the above function for all identified postal codes

M5A
M9A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [17]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['PostalCode'] = Toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

In [18]:
Toronto_grouped = Toronto_onehot.groupby('PostalCode').mean().reset_index()

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
PostalCode_venues_sorted = pd.DataFrame(columns=columns)
PostalCode_venues_sorted['PostalCode'] = Toronto_grouped['PostalCode']

for ind in np.arange(Toronto_grouped.shape[0]):
    PostalCode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

In [21]:
# set number of clusters
kclusters = 7

Toronto_grouped_clustering = Toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
PostalCode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(PostalCode_venues_sorted.set_index('PostalCode'), on='PostalCode', how = 'right')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,6,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Dessert Shop
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,6,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant,Café,Bakery,Diner,Italian Restaurant,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,6,Café,Coffee Shop,Restaurant,Clothing Store,Hotel,Italian Restaurant,Beer Bar,American Restaurant,Cosmetics Shop,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Other Great Outdoors,Trail,Pub,Neighborhood,Health Food Store,Doner Restaurant,Dog Run,Donut Shop,Dance Studio,Discount Store
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,6,Coffee Shop,Steakhouse,Cocktail Bar,Café,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Farmers Market,Bistro


In [22]:
Toronto_merged.head(45)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,6,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Dessert Shop
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,6,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant,Café,Bakery,Diner,Italian Restaurant,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,6,Café,Coffee Shop,Restaurant,Clothing Store,Hotel,Italian Restaurant,Beer Bar,American Restaurant,Cosmetics Shop,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Other Great Outdoors,Trail,Pub,Neighborhood,Health Food Store,Doner Restaurant,Dog Run,Donut Shop,Dance Studio,Discount Store
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,6,Coffee Shop,Steakhouse,Cocktail Bar,Café,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Farmers Market,Bistro
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,6,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Ice Cream Shop,Burger Joint,Chinese Restaurant,Salad Place,Spa,Juice Bar
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,6,Grocery Store,Café,Park,Restaurant,Baby Store,Athletics & Sports,Candy Store,Diner,Italian Restaurant,Nightclub
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,6,Coffee Shop,Café,Steakhouse,Thai Restaurant,Salad Place,Burger Joint,Bar,Bakery,Sushi Restaurant,Asian Restaurant
9,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259,6,Bakery,Pharmacy,Supermarket,Bar,Café,Middle Eastern Restaurant,Art Gallery,Bank,Park,Smoke Shop
10,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,6,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Scenic Lookout,Brewery,Restaurant,Fried Chicken Joint,Bar


In [23]:
address = 'Toronto, ON' #create a user agent

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['PostalCode'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The dominant group is group 6, which seems to reprsent the "downtown mix" : coffee shops, bars, and restaurants. This mix dominates downtown Toronto as well as postal codes near the urban core. The other clusters are very small and at the very edge of the Toronto area. Other areas include the "park group" with two widely seperated members on the edges of town. Other "clusters" are more standouts, with substantially different top 10s. Further clustering is likely to be ineffective, only creating sub clusters of the downtown mix that do not do much to further explain the diversity of the groups.